In [ ]:
# Instalação de bibliotecas
!pip install protobuf pandas openpyxl matplotlib
!apt-get install -y lsof protobuf-compiler

In [ ]:
# Interface para upload e seleção de arquivos via widgets
import ipywidgets as widgets
from google.colab import files
from IPython.display import display

print('Clique abaixo para enviar os arquivos .proto, .pb, .txt (HALs) e .desc, se houver:')
uploaded = files.upload()

arquivo_paths = list(uploaded.keys())
multi_select = widgets.SelectMultiple(
    options=arquivo_paths,
    description='Selecionar:',
    rows=10,
    layout=widgets.Layout(width='90%')
)
display(multi_select)

In [ ]:
# Compilação dos arquivos .proto para módulos Python
import os
for nome in arquivo_paths:
    if nome.endswith('.proto'):
        os.system(f"protoc --python_out=. {nome}")
        print(f"Compilado: {nome}")

In [ ]:
# Leitura de arquivos .pb com base no módulo compilado
import importlib.util

# Substitua 'YourMessage' pelo nome real da mensagem do seu .proto
for nome in arquivo_paths:
    if nome.endswith('_pb2.py'):
        spec = importlib.util.spec_from_file_location("pb_module", nome)
        pb_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(pb_module)
        for pb_file in arquivo_paths:
            if pb_file.endswith('.pb'):
                with open(pb_file, 'rb') as f:
                    try:
                        mensagem = pb_module.YourMessage()
                        mensagem.ParseFromString(f.read())
                        print(f"{pb_file}: OK\n{mensagem}")
                    except Exception as e:
                        print(f"Erro ao ler {pb_file}: {e}")

In [ ]:
# Análise de arquivos HAL e exportação de status
import pandas as pd
import matplotlib.pyplot as plt

resultados = []
for nome in arquivo_paths:
    if nome.endswith('.txt'):
        with open(nome, 'r', encoding='utf-8', errors='ignore') as f:
            conteudo = f.read()
            status = 'OK'
            if 'failed' in conteudo.lower() or 'exit code' in conteudo.lower():
                status = 'Falha detectada'
            resultados.append({"Arquivo": nome, "Status": status})

df = pd.DataFrame(resultados)
df.to_excel('resultado_HALs_lote.xlsx', index=False)
df

In [ ]:
# Visualização gráfica das falhas HAL
status_counts = df['Status'].value_counts()
plt.figure(figsize=(6, 4))
status_counts.plot(kind='bar', color=['green', 'red'])
plt.title('Resumo de Falhas HAL por Lote')
plt.xlabel('Status')
plt.ylabel('Quantidade de Arquivos')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()